In [1]:
import torch 
import json,pickle,math
import pandas as pd
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [2]:
LSM = pickle.load(open('./davis_ligand_similarity_matrix.pkl', 'rb'))
PSM = pickle.load(open('./davis_protein_similarity_matrix.pkl', 'rb'))
full_df = pd.read_csv(open('./davis_all_pairs.csv','r'))

In [3]:
# df

In [4]:
SMILES = json.load(open('./data/DAVIS/SMILES.txt'))
TARGETS = json.load(open('./data/DAVIS/target_seq.txt'))
SMILES=list(SMILES.values())
TARGETS=list(TARGETS.values())

In [5]:
all_9_folds={}
for i in [0,1,2]:
    for j in [0,1,2]:
        file_name = 'fold' +str(i) +str(j) 
        
        temp = open('./data/davis/DAVIS_9_FOLDS/' + file_name +'.pkl', 'rb')
        new_df = pd.read_pickle(temp)
        all_9_folds.update({file_name:new_df})
        temp.close()
        

In [6]:
def create_davis_test_train(test_fold_number,all_9_folds):
    test_protein_fold_id = test_fold_number[0]
    test_ligand_fold_id = test_fold_number[1]
    test_set = pd.DataFrame(columns = full_df.columns)
    train_set = pd.DataFrame(columns= full_df.columns)
    for i in [0,1,2]:
        for j in [0,1,2]:
            fold_name = 'fold' + str(i) + str(j)
            df = all_9_folds[fold_name]
            
            if str(i) == test_protein_fold_id:
                print("test->", str(i), str(j))
                test_set = pd.concat([test_set, df.copy()], ignore_index=True)
                
            else:
                print("train->", str(i), str(j))
                train_set = pd.concat([train_set, df.copy()], ignore_index=True)
                
                
    return train_set, test_set


# Create train test split on these 9 folds
## fold_number is the id of fold. For example, test = fold00, train = fold 11,22,12,21

In [7]:
fold_number = '2x'

In [8]:
train, test = create_davis_test_train(test_fold_number=fold_number, all_9_folds=all_9_folds)

train-> 0 0
train-> 0 1
train-> 0 2
train-> 1 0
train-> 1 1
train-> 1 2
test-> 2 0
test-> 2 1
test-> 2 2


In [10]:
test

,SMILES,Target Sequence,Label,drug_encoding,target_encoding
0,CCC1C(=O)N(C2=CN=C(N=C2N1C3CCCC3)NC4=C(C=C(C=C...,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,5.552842,"[C, C, C, 1, C, (, =, O, ), N, (, C, 2, =, C, ...","[M, K, K, F, F, D, S, R, R, E, Q, G, G, S, G, ..."
1,CN1C2=C(C=C(C=C2)OC3=CC(=NC=C3)C4=NC=C(N4)C(F)...,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,5.000000,"[C, N, 1, C, 2, =, C, (, C, =, C, (, C, =, C, ...","[M, K, K, F, F, D, S, R, R, E, Q, G, G, S, G, ..."
2,C1CC1CONC(=O)C2=C(C(=C(C=C2)F)F)NC3=C(C=C(C=C3...,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,5.000000,"[C, 1, C, C, 1, C, O, N, C, (, =, O, ), C, 2, ...","[M, K, K, F, F, D, S, R, R, E, Q, G, G, S, G, ..."
3,CC1=CC2=C(N1)C=CC(=C2F)OC3=NC=NC4=CC(=C(C=C43)...,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,5.000000,"[C, C, 1, =, C, C, 2, =, C, (, N, 1, ), C, =, ...","[M, K, K, F, F, D, S, R, R, E, Q, G, G, S, G, ..."
4,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=NC(=NC(...,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...,5.000000,"[C, C, 1, =, C, (, C, (, =, C, C, =, C, 1, ), ...","[M, K, K, F, F, D, S, R, R, E, Q, G, G, S, G, ..."
...,...,...,...,...,...
9991,CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...,MDDKDIDKELRQKLNFSYCEETEIEGQKKVEESREASSQTPEKGEV...,5.000000,"[C, N, 1, C, =, N, C, 2, =, C, 1, C, =, C, (, ...","[M, D, D, K, D, I, D, K, E, L, R, Q, K, L, N, ..."
9992,CC(C)S(=O)(=O)C1=CC=CC=C1NC2=NC(=NC=C2Cl)NC3=C...,MDDKDIDKELRQKLNFSYCEETEIEGQKKVEESREASSQTPEKGEV...,5.000000,"[C, C, (, C, ), S, (, =, O, ), (, =, O, ), C, ...","[M, D, D, K, D, I, D, K, E, L, R, Q, K, L, N, ..."
9993,C1=CC(=CC(=C1)O)C2=NC3=C(N=C2C4=CC(=CC=C4)O)N=...,MDDKDIDKELRQKLNFSYCEETEIEGQKKVEESREASSQTPEKGEV...,5.000000,"[C, 1, =, C, C, (, =, C, C, (, =, C, 1, ), O, ...","[M, D, D, K, D, I, D, K, E, L, R, Q, K, L, N, ..."
9994,CC1=CN=C(N=C1NC2=CC(=CC=C2)S(=O)(=O)NC(C)(C)C)...,MDDKDIDKELRQKLNFSYCEETEIEGQKKVEESREASSQTPEKGEV...,5.221849,"[C, C, 1, =, C, N, =, C, (, N, =, C, 1, N, C, ...","[M, D, D, K, D, I, D, K, E, L, R, Q, K, L, N, ..."


# To ensure that there are no common targets or drugs in train and test


In [11]:
test_smiles = list(test['SMILES'])
test_targets = list(test['Target Sequence'])
train_smiles = list(train['SMILES'])
train_targets = list(train['Target Sequence'])

# for i in test_smiles:
#     if i in train_smiles:
#         print("common entity present")
for i in test_targets:
    if i in train_targets:
        print("common entity present")


# Creating outer products for test set

In [12]:
outer_test_prods = []
for i,row in test.iterrows():
#     print(i)
    smi = row['SMILES']
    seq = row['Target Sequence']
    target_id = TARGETS.index(seq)
    smi_id = SMILES.index(smi)
    ki=LSM[smi_id]
    kj=PSM[target_id]
    ki_x_kj = np.outer(ki,kj)
    outer_test_prods.append([ki_x_kj])
outer_test_prods = np.array(outer_test_prods)
print(np.shape(outer_test_prods))

(9996, 1, 68, 442)


# creating outer products for train set

In [13]:
outer_train_prods = []
for i,row in train.iterrows():
#     print(i)
    smi = row['SMILES']
    seq = row['Target Sequence']
    target_id = TARGETS.index(seq)
    smi_id = SMILES.index(smi)
    ki=LSM[smi_id]
    kj=PSM[target_id]
    ki_x_kj = np.outer(ki,kj)
    outer_train_prods.append([ki_x_kj])
outer_train_prods = np.array(outer_train_prods)
print(np.shape(outer_train_prods))

(20060, 1, 68, 442)


In [14]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 20
# num_classes = 10
batch_size = 32
learning_rate = 0.001

In [15]:
class custom_dataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, outer_prods, transform=None):
#         self.df = pd.read_csv(open(csv_file))
        self.df = dataframe
#         self.root_dir = root_dir
        self.transform = transform
        self.outer_prods = outer_prods
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        output = {'outer_product': self.outer_prods[idx] , 'Label':self.df.iloc[idx]['Label']}
        return output

In [16]:
train_dataset = custom_dataset(dataframe = train, outer_prods = outer_train_prods)
test_dataset = custom_dataset(dataframe = test, outer_prods = outer_test_prods)


In [17]:
train_loader= torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader= torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [18]:
print(len(train_loader)*32, len(test_loader)*32)

20064 10016


In [19]:
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1,32, 5).double()
        self.pool1 = nn.MaxPool2d(2,2).double()
        self.conv2 = nn.Conv2d(32,18,3).double()
        self.pool2 = nn.MaxPool2d(2,2).double()
        self.fc1 = nn.Linear(18*15*108, 128).double()
        self.fc2 = nn.Linear(128,1).double()
        self.dropout = nn.Dropout(0.1).double()
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1,18*15*108)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        
        return x
    

In [20]:
# for i in test_loader:
#     a = i['outer_product']
#     b= i['Label']
#     break
# conv1 = nn.Conv2d(1,32,5).double()
# pool = nn.MaxPool2d(2,2).double()
# conv2 = nn.Conv2d(32,18,3).double()
# fc1 = nn.Linear(18*15*108, 128).double()
# fc2 = nn.Linear(128,1).double()
# dropout = nn.Dropout(0.1).double()
# x= conv1(a)
# print(x.shape)
# x = pool(x)
# print(x.shape)
# x= conv2(x)
# print(x.shape)
# x = pool(x)
# print(x.shape)
# x = x.view(-1,18*15*108)
# print(x.shape)
# x = dropout(x)
# print(x.shape)
# x = fc1(x)
# print(x.shape)
# x = fc2(x)
# print(x.shape)

In [21]:
model = ConvNet().to(device)

In [22]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# Evaluation metrics

In [23]:
def rmse(y,f):
    rmse = math.sqrt(((y - f)**2).mean(axis=0))
    return rmse
def mse(y,f):
    mse = ((y - f)**2).mean(axis=0)
    return mse
def pearson(y,f):
    rp = np.corrcoef(y, f)[0,1]
    return rp
from lifelines.utils import concordance_index
def ci(y,f):
    return concordance_index(y,f)

In [24]:
def predicting(model, device, test_loader):
    model.eval()
    total_preds = np.array([])
    total_labels = np.array([])
    with torch.no_grad():
        for i in test_loader:
            images = i['outer_product']
            labels = i['Label']
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images) 
            outputs = outputs.cpu().detach().numpy().flatten()
            labels =labels.cpu().detach().numpy().flatten()
            total_preds = np.concatenate([total_preds, outputs])
            total_labels = np.concatenate([total_labels, labels])
    
    model.train()
    return total_labels, total_preds

# Train the model

In [25]:
model_file_name = './SAVED_MODELS/best_sim-CNN-DTA_davis_fold' + fold_number +  '.model'
result_file_name = './SAVED_MODELS/best_result_sim-CNNDTA_davis_fold'+fold_number + '.csv'

In [26]:
# Train the model
best_mse = 1000
best_ci = 0

total_step = len(train_loader)
for epoch in range(num_epochs):
    c=0
    for i in train_loader:
        c=c+1
        images = i['outer_product']
        labels = i['Label']
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.flatten(), labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, c, total_step, loss.item()))
    
    # taking best model so far
    G,P = predicting(model, device, test_loader)
    ret = [rmse(G, P), mse(G, P), pearson(G, P), ci(G, P)]
    if ret[1] < best_mse:
        torch.save(model.state_dict(), model_file_name)
        with open(result_file_name, 'w') as f:
            f.write(','.join(map(str, ret)))
        best_epoch = epoch+1
        best_mse = ret[1]
        best_ci = ret[-1]
        best_r = ret[2]
        
        print('rmse improved at epoch ', best_epoch,
                      '; best_mse,best_ci,best_r:', best_mse, best_ci,best_r)
        
        

/home2/kanakala.ganesh/miniconda3/envs/geometric/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch [1/20], Step [1/627], Loss: 31.4504
Epoch [1/20], Step [2/627], Loss: 19.5914
Epoch [1/20], Step [3/627], Loss: 6.2409
Epoch [1/20], Step [4/627], Loss: 2.5250
Epoch [1/20], Step [5/627], Loss: 11.1518
Epoch [1/20], Step [6/627], Loss: 5.3322
Epoch [1/20], Step [7/627], Loss: 0.8963
Epoch [1/20], Step [8/627], Loss: 2.7817
Epoch [1/20], Step [9/627], Loss: 2.9538
Epoch [1/20], Step [10/627], Loss: 5.9310
Epoch [1/20], Step [11/627], Loss: 2.9157
Epoch [1/20], Step [12/627], Loss: 1.5769
Epoch [1/20], Step [13/627], Loss: 0.5359
Epoch [1/20], Step [14/627], Loss: 1.7473
Epoch [1/20], Step [15/627], Loss: 1.5474
Epoch [1/20], Step [16/627], Loss: 2.7055
Epoch [1/20], Step [17/627], Loss: 2.5188
Epoch [1/20], Step [18/627], Loss: 2.1426
Epoch [1/20], Step [19/627], Loss: 0.8611
Epoch [1/20], Step [20/627], Loss: 0.3345
Epoch [1/20], Step [21/627], Loss: 1.1230
Epoch [1/20], Step [22/627], Loss: 0.7509
Epoch [1/20], Step [23/627], Loss: 1.7591
Epoch [1/20], Step [24/627], Loss: 2.418

Epoch [1/20], Step [196/627], Loss: 0.6967
Epoch [1/20], Step [197/627], Loss: 0.4919
Epoch [1/20], Step [198/627], Loss: 0.6139
Epoch [1/20], Step [199/627], Loss: 0.6775
Epoch [1/20], Step [200/627], Loss: 0.3781
Epoch [1/20], Step [201/627], Loss: 1.1015
Epoch [1/20], Step [202/627], Loss: 0.8906
Epoch [1/20], Step [203/627], Loss: 0.9561
Epoch [1/20], Step [204/627], Loss: 0.7127
Epoch [1/20], Step [205/627], Loss: 0.6991
Epoch [1/20], Step [206/627], Loss: 1.1278
Epoch [1/20], Step [207/627], Loss: 0.3116
Epoch [1/20], Step [208/627], Loss: 0.2528
Epoch [1/20], Step [209/627], Loss: 0.7551
Epoch [1/20], Step [210/627], Loss: 0.8686
Epoch [1/20], Step [211/627], Loss: 0.3915
Epoch [1/20], Step [212/627], Loss: 0.4003
Epoch [1/20], Step [213/627], Loss: 0.2845
Epoch [1/20], Step [214/627], Loss: 0.6475
Epoch [1/20], Step [215/627], Loss: 1.2669
Epoch [1/20], Step [216/627], Loss: 2.1014
Epoch [1/20], Step [217/627], Loss: 0.4406
Epoch [1/20], Step [218/627], Loss: 1.0686
Epoch [1/20

Epoch [1/20], Step [388/627], Loss: 0.2691
Epoch [1/20], Step [389/627], Loss: 0.6170
Epoch [1/20], Step [390/627], Loss: 0.4998
Epoch [1/20], Step [391/627], Loss: 0.3463
Epoch [1/20], Step [392/627], Loss: 0.6059
Epoch [1/20], Step [393/627], Loss: 0.6573
Epoch [1/20], Step [394/627], Loss: 0.9964
Epoch [1/20], Step [395/627], Loss: 0.6863
Epoch [1/20], Step [396/627], Loss: 0.1565
Epoch [1/20], Step [397/627], Loss: 0.6478
Epoch [1/20], Step [398/627], Loss: 0.4411
Epoch [1/20], Step [399/627], Loss: 0.3092
Epoch [1/20], Step [400/627], Loss: 0.5386
Epoch [1/20], Step [401/627], Loss: 0.8444
Epoch [1/20], Step [402/627], Loss: 0.4260
Epoch [1/20], Step [403/627], Loss: 0.5174
Epoch [1/20], Step [404/627], Loss: 0.9732
Epoch [1/20], Step [405/627], Loss: 0.5040
Epoch [1/20], Step [406/627], Loss: 1.4217
Epoch [1/20], Step [407/627], Loss: 0.7794
Epoch [1/20], Step [408/627], Loss: 0.6793
Epoch [1/20], Step [409/627], Loss: 0.4231
Epoch [1/20], Step [410/627], Loss: 0.2750
Epoch [1/20

Epoch [1/20], Step [580/627], Loss: 0.3203
Epoch [1/20], Step [581/627], Loss: 0.3754
Epoch [1/20], Step [582/627], Loss: 0.6231
Epoch [1/20], Step [583/627], Loss: 0.8504
Epoch [1/20], Step [584/627], Loss: 0.3872
Epoch [1/20], Step [585/627], Loss: 0.5698
Epoch [1/20], Step [586/627], Loss: 0.5910
Epoch [1/20], Step [587/627], Loss: 0.3922
Epoch [1/20], Step [588/627], Loss: 0.6934
Epoch [1/20], Step [589/627], Loss: 0.5749
Epoch [1/20], Step [590/627], Loss: 1.0668
Epoch [1/20], Step [591/627], Loss: 0.5068
Epoch [1/20], Step [592/627], Loss: 0.6818
Epoch [1/20], Step [593/627], Loss: 0.2991
Epoch [1/20], Step [594/627], Loss: 0.3283
Epoch [1/20], Step [595/627], Loss: 0.6483
Epoch [1/20], Step [596/627], Loss: 0.7957
Epoch [1/20], Step [597/627], Loss: 0.7192
Epoch [1/20], Step [598/627], Loss: 0.5210
Epoch [1/20], Step [599/627], Loss: 0.6543
Epoch [1/20], Step [600/627], Loss: 0.4970
Epoch [1/20], Step [601/627], Loss: 0.5824
Epoch [1/20], Step [602/627], Loss: 1.0442
Epoch [1/20

...

Epoch [19/20], Step [349/627], Loss: 0.0959
Epoch [19/20], Step [350/627], Loss: 0.1037
Epoch [19/20], Step [351/627], Loss: 0.1412
Epoch [19/20], Step [352/627], Loss: 0.2883
Epoch [19/20], Step [353/627], Loss: 0.0759
Epoch [19/20], Step [354/627], Loss: 0.1583
Epoch [19/20], Step [355/627], Loss: 0.0556
Epoch [19/20], Step [356/627], Loss: 0.4582
Epoch [19/20], Step [357/627], Loss: 0.0546
Epoch [19/20], Step [358/627], Loss: 0.1200
Epoch [19/20], Step [359/627], Loss: 0.1550
Epoch [19/20], Step [360/627], Loss: 0.1412
Epoch [19/20], Step [361/627], Loss: 0.1935
Epoch [19/20], Step [362/627], Loss: 0.2023
Epoch [19/20], Step [363/627], Loss: 0.1399
Epoch [19/20], Step [364/627], Loss: 0.1000
Epoch [19/20], Step [365/627], Loss: 0.2092
Epoch [19/20], Step [366/627], Loss: 0.0757
Epoch [19/20], Step [367/627], Loss: 0.1373
Epoch [19/20], Step [368/627], Loss: 0.0725
Epoch [19/20], Step [369/627], Loss: 0.2725
Epoch [19/20], Step [370/627], Loss: 0.1751
Epoch [19/20], Step [371/627], L

Epoch [19/20], Step [538/627], Loss: 0.1117
Epoch [19/20], Step [539/627], Loss: 0.0923
Epoch [19/20], Step [540/627], Loss: 0.0902
Epoch [19/20], Step [541/627], Loss: 0.1955
Epoch [19/20], Step [542/627], Loss: 0.1048
Epoch [19/20], Step [543/627], Loss: 0.1078
Epoch [19/20], Step [544/627], Loss: 0.1563
Epoch [19/20], Step [545/627], Loss: 0.0690
Epoch [19/20], Step [546/627], Loss: 0.2377
Epoch [19/20], Step [547/627], Loss: 0.1074
Epoch [19/20], Step [548/627], Loss: 0.0950
Epoch [19/20], Step [549/627], Loss: 0.1764
Epoch [19/20], Step [550/627], Loss: 0.0475
Epoch [19/20], Step [551/627], Loss: 0.3124
Epoch [19/20], Step [552/627], Loss: 0.1219
Epoch [19/20], Step [553/627], Loss: 0.0491
Epoch [19/20], Step [554/627], Loss: 0.1560
Epoch [19/20], Step [555/627], Loss: 0.0936
Epoch [19/20], Step [556/627], Loss: 0.0330
Epoch [19/20], Step [557/627], Loss: 0.1314
Epoch [19/20], Step [558/627], Loss: 0.3192
Epoch [19/20], Step [559/627], Loss: 0.2797
Epoch [19/20], Step [560/627], L

Epoch [20/20], Step [100/627], Loss: 0.0850
Epoch [20/20], Step [101/627], Loss: 0.1769
Epoch [20/20], Step [102/627], Loss: 0.1458
Epoch [20/20], Step [103/627], Loss: 0.1076
Epoch [20/20], Step [104/627], Loss: 0.1290
Epoch [20/20], Step [105/627], Loss: 0.1743
Epoch [20/20], Step [106/627], Loss: 0.0932
Epoch [20/20], Step [107/627], Loss: 0.1286
Epoch [20/20], Step [108/627], Loss: 0.1507
Epoch [20/20], Step [109/627], Loss: 0.0991
Epoch [20/20], Step [110/627], Loss: 0.1042
Epoch [20/20], Step [111/627], Loss: 0.1542
Epoch [20/20], Step [112/627], Loss: 0.0768
Epoch [20/20], Step [113/627], Loss: 0.1391
Epoch [20/20], Step [114/627], Loss: 0.2723
Epoch [20/20], Step [115/627], Loss: 0.2640
Epoch [20/20], Step [116/627], Loss: 0.2036
Epoch [20/20], Step [117/627], Loss: 0.0507
Epoch [20/20], Step [118/627], Loss: 0.1007
Epoch [20/20], Step [119/627], Loss: 0.1240
Epoch [20/20], Step [120/627], Loss: 0.1255
Epoch [20/20], Step [121/627], Loss: 0.0545
Epoch [20/20], Step [122/627], L

Epoch [20/20], Step [289/627], Loss: 0.0747
Epoch [20/20], Step [290/627], Loss: 0.4962
Epoch [20/20], Step [291/627], Loss: 0.0364
Epoch [20/20], Step [292/627], Loss: 0.1106
Epoch [20/20], Step [293/627], Loss: 0.0873
Epoch [20/20], Step [294/627], Loss: 0.1224
Epoch [20/20], Step [295/627], Loss: 0.1895
Epoch [20/20], Step [296/627], Loss: 0.1404
Epoch [20/20], Step [297/627], Loss: 0.0423
Epoch [20/20], Step [298/627], Loss: 0.1440
Epoch [20/20], Step [299/627], Loss: 0.0602
Epoch [20/20], Step [300/627], Loss: 0.1055
Epoch [20/20], Step [301/627], Loss: 0.0879
Epoch [20/20], Step [302/627], Loss: 0.0553
Epoch [20/20], Step [303/627], Loss: 0.1055
Epoch [20/20], Step [304/627], Loss: 0.0872
Epoch [20/20], Step [305/627], Loss: 0.1027
Epoch [20/20], Step [306/627], Loss: 0.1000
Epoch [20/20], Step [307/627], Loss: 0.0325
Epoch [20/20], Step [308/627], Loss: 0.1235
Epoch [20/20], Step [309/627], Loss: 0.3196
Epoch [20/20], Step [310/627], Loss: 0.1658
Epoch [20/20], Step [311/627], L

Epoch [20/20], Step [478/627], Loss: 0.0459
Epoch [20/20], Step [479/627], Loss: 0.2462
Epoch [20/20], Step [480/627], Loss: 0.1158
Epoch [20/20], Step [481/627], Loss: 0.0727
Epoch [20/20], Step [482/627], Loss: 0.2590
Epoch [20/20], Step [483/627], Loss: 0.1453
Epoch [20/20], Step [484/627], Loss: 0.2572
Epoch [20/20], Step [485/627], Loss: 0.1088
Epoch [20/20], Step [486/627], Loss: 0.3081
Epoch [20/20], Step [487/627], Loss: 0.0699
Epoch [20/20], Step [488/627], Loss: 0.0848
Epoch [20/20], Step [489/627], Loss: 0.1954
Epoch [20/20], Step [490/627], Loss: 0.0913
Epoch [20/20], Step [491/627], Loss: 0.1117
Epoch [20/20], Step [492/627], Loss: 0.1593
Epoch [20/20], Step [493/627], Loss: 0.1085
Epoch [20/20], Step [494/627], Loss: 0.1241
Epoch [20/20], Step [495/627], Loss: 0.0698
Epoch [20/20], Step [496/627], Loss: 0.0825
Epoch [20/20], Step [497/627], Loss: 0.0548
Epoch [20/20], Step [498/627], Loss: 0.1911
Epoch [20/20], Step [499/627], Loss: 0.1475
Epoch [20/20], Step [500/627], L

In [27]:
best_epoch

1

In [28]:
model.eval()
# eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
total_preds = np.array([])
total_labels = np.array([])
with torch.no_grad():
    correct = 0
    total = 0
    for i in test_loader:
        images = i['outer_product']
        labels = i['Label']
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images) 
        outputs = outputs.cpu().detach().numpy().flatten()
        labels =labels.cpu().detach().numpy().flatten()
        total_preds = np.concatenate([total_preds, outputs])
        total_labels = np.concatenate([total_labels, labels])
#         total_preds = torch.cat(total_preds, outputs.cpu(), 0 )
#         total_labels = torch.cat(total_labels, labels.cpu(), 0)
#         break

In [29]:
G,P = total_labels, total_preds

In [30]:
rmse(G,P)

1.0997996272101132

In [31]:
mse(G,P)

1.209559220011504

In [32]:
pearson(G,P)

0.2238972960931255

In [33]:
ci(G,P)

0.6104435022808651